## VA_model 2
### copy from ~/Desktop/P/pkl_to_csv.ipynb

In [1]:
from pathlib import Path as P
import os, pickle

In [2]:
"load dataset.pkl"
base_dir = P("total_dataset")
assert base_dir.is_dir()

file_dir = base_dir/"dataset.pkl"

with open(file_dir,"rb") as f:
    dataset = pickle.load(f)

len(dataset)

23510

In [3]:
dataset[0]

('伴侶：臭貓貓 我玩遊戲的時候一直煩我，罵他還一直蹭我😡\n（還是伴侶）\n伴侶：我早上幫他乾洗澡後他就不讓我摸摸了😭', 'anger', '😡')

In [4]:
label2int = {
    "sadness": 0,
    "joy": 1,
    "love": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5
}

### "4 columns, (text, emotion, label (int), emoji"

## Shuffle the data

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset

### We use pickle directly, since csv keep running into key error

In [6]:
"Try from reading pure python list and manipulate with dataloader"
import pickle

with open(base_dir/"dataset.pkl", "rb") as f:
    data_list = pickle.load(f)

data_list[0], len(data_list)

(('伴侶：臭貓貓 我玩遊戲的時候一直煩我，罵他還一直蹭我😡\n（還是伴侶）\n伴侶：我早上幫他乾洗澡後他就不讓我摸摸了😭', 'anger', '😡'),
 23510)

In [7]:
raw_data = [(ele[0], label2int[ele[1]]) for ele in data_list]

raw_data[0]

('伴侶：臭貓貓 我玩遊戲的時候一直煩我，罵他還一直蹭我😡\n（還是伴侶）\n伴侶：我早上幫他乾洗澡後他就不讓我摸摸了😭', 3)

In [8]:
class D_Dataset(Dataset):

    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        return (self.data[idx][0], self.data[idx][1])

    def __len__(self):
        return len(self.data)

In [9]:
d = D_Dataset(raw_data)

In [10]:
d[0]

('伴侶：臭貓貓 我玩遊戲的時候一直煩我，罵他還一直蹭我😡\n（還是伴侶）\n伴侶：我早上幫他乾洗澡後他就不讓我摸摸了😭', 3)

In [11]:
dl = DataLoader(d,shuffle=False, batch_size=64)

In [12]:
next(iter(dl))

[('伴侶：臭貓貓 我玩遊戲的時候一直煩我，罵他還一直蹭我😡\n（還是伴侶）\n伴侶：我早上幫他乾洗澡後他就不讓我摸摸了😭',
  '😡\n\n好一天又罷工，我的腰好不爭氣',
  '有一種，終於結束的感覺了嗎？🤔\n\n在我之前的噗文裡，有一個叫「家蓉緋緋」的，\n她當時在我還就職在寵物學校裡時，就讓我發現他是個很愛裝「我是無辜無害的」女人。\n\n她曾跟我說：「以前當學生時，都被其他人霸凌」\n而我後來也跟他說：「你被霸凌是應該的，因為你處處要跟每個人好，你每個人都拼命想討好，所以你會被霸凌是正常的！」\n\n後來我離開寵美學校後，我也不再與他們聯絡，打死我都不會再聯絡的😡\n\n唯一一次的聯絡，是在去年傳了這個訊息給我😡\n\n後來這個截圖我拿去詢問較好的同行朋友，每個人都在說：「這個人是神經病嗎？他有問題是不是？分不清責任歸屬？」\n\n為何我會說這事情彷彿告一段落的原因是⋯這女人因為朋友關係，又退出所有學校相關的群組了👍',
  '[日韓擔的最怒] [知韓知日，學優明缺]\n@enfihu - Kakao 想戰 Google ? 也是有影響到數位音樂平台付費喔！ 我有在ptt韓...\n/ 前情提要\n—\n我開始覺得KBS 對不朽名曲/柳喜烈寫生簿/開放音樂會演出內容，管非韓國IP、是否更換/釋出多角度影片的『彈性標準』感到火大😡🤬\n外飯的瀏覽權不重要嗎？ - 當然也有Content ID /版權等等因素\n把我們當提款機嗎！',
  '中國「飛彈首次穿越台灣」 國防部告訴你為何不擊落、不公布、沒警報｜蘋果新聞網｜蘋果日報\n真的有飛彈從上空飛過去 從上面飛過去這就過分了吧😡',
  '薯的第四十七/八天\n\n回家果然沒什麼動草 然後不知道為什麼一直把飼料推出去😡\n馬上就拆盈盈的葉多\n雖然顏色沒有很翠綠了 但真的葉多！幾乎整包都葉子🤣薯的接受度蠻高的好棒\n而且可能因為都是葉子 所以他講他也有多吃一點梗\n明天會再放一點花穗 因為感覺他對葉子要求很高⋯明明就一堆在那邊還是不想吃\n\n今天幫她大掃除 真的是亂尿尿到連巧拼上都有\n但整個換完後他到目前都沒有亂尿尿了\n所以可能真的不能偷懶要幫他每天換便盆\n最近在考慮要不要請天友做墊子\n這樣薯趴在那邊吃草時才不會碰到尿尿 還在觀察他這兩天抓癢的情況 就是眼周跟下半身居多 目前只先推測是冷氣吹到太

### play around the model here

In [13]:
from EMO_AI.all import *

base_model = get_model(pretrained=False)
base_model.classifier

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Sequential(
  (0): Dropout(p=0.05, inplace=False)
  (1): Linear(in_features=768, out_features=768, bias=True)
  (2): Mish()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=768, out_features=6, bias=True)
)

In [15]:
output_ = base_model(convert_text_to_tensor("This love"))
output_.shape, output_

(torch.Size([1, 6]),
 tensor([[ 0.0193, -0.0264, -0.0235, -0.0358,  0.0220, -0.0137]],
        grad_fn=<AddmmBackward0>))

### Process the data to the right shape (read from DataLoader)

In [16]:
class TokenizersCollateFn:
    def __init__(self, max_tokens=512):
        """
        from tokenizers import ByteLevelBPETokenizer
        from tokenizers.processors import BertProcessing
        ## RoBERTa uses BPE tokenizer similar to GPT
        t = ByteLevelBPETokenizer(
            "tokenizer/vocab.json",
            "tokenizer/merges.txt"
        )
        t._tokenizer.post_processor = BertProcessing(
            ("</s>", t.token_to_id("</s>")),
            ("<s>", t.token_to_id("<s>")),
        )
        t.enable_truncation(max_tokens)
        t.enable_padding(length=max_tokens, pad_id=t.token_to_id("<pad>"))
        self.tokenizer = t"""
        self.tokenizer = get_tokenizer()

    def __call__(self, batch):
        encoded = self.tokenizer.encode_batch([x[0] for x in batch])
        sequences_padded = torch.tensor([enc.ids for enc in encoded])
        attention_masks_padded = torch.tensor([enc.attention_mask for enc in encoded])
        labels = torch.tensor([x[1] for x in batch])
        
        return (sequences_padded, attention_masks_padded), labels

In [17]:
DL = DataLoader(d,shuffle=False, batch_size=64, collate_fn=TokenizersCollateFn())

"(x,atten_x, y)"
next(iter(DL))

((tensor([[    0, 48341, 20024,  ...,     1,     1,     1],
          [    0, 18636,  5543,  ...,     1,     1,     1],
          [    0, 44636, 23171,  ..., 48537, 10278,     2],
          ...,
          [    0, 47876,  3602,  ...,     1,     1,     1],
          [    0, 48548,  9085,  ...,     1,     1,     1],
          [    0,   250,   246,  ...,     1,     1,     1]]),
  tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]])),
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]))

In [27]:
(x,at), y = tmp
x.shape, at.shape, y.shape

(torch.Size([64, 512]), torch.Size([64, 512]), torch.Size([64]))

In [14]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
"""
Two different way to approach this problem
Proposal 1: DummyModel()
x: text_tensor, y: emotion (int, the 6-label thing)
train the model, as usual, on text and label
learn the VA layer, F layer by training
check the output (y), to see if the VA layer, F layer helps to predict the emotion better

Attention: 
the x it trains on should be A-B-A conversation 
to learn the emotion change

Proposal 2:
manually label the VA, F of each set of data
, and train the model with label of emotion(int), VA(tensor), F(tensor)

option 1: assume VA, F already affect the conversation undering hood
...

"""

In [29]:
"""Proposal 1"""
class DummyModel(nn.Module):
    def __init__(self, base_model):
        self.base_model = base_model

        "feed s into f"
        "output: f"
        self.s_f_layer = nn.Linear()

        "output: s"
        self.s_layer = nn.Linear()

        "feed x, s, va_old to get va_new"
        "output: va_new"
        self.va_layer = nn.Linear()


    def forward(self, x, va_old):
        "input VA_old and text(string)"
        "tensor: x:; tensor: va_old"
        "tensor: emotion"
        emotion = self.base_model(x)
        "tensor: s: affected by x and va_old"
        # check the dim again!
        s_input = torch.concat((emotion, va_old), dim=1)
        s = self.s_layer(s_input)

        f = self.s_f_layer(s)

        va_input = torch.concat((va_old, s, f), dim=1)
        va_new = self.va_layer(va_input)
        
        "return VA_new"
        return va_new
